In [3]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [4]:

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS demo 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [5]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [6]:
import os,glob
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

C:\Users\Zain\dataengineer\apache-cassandra


In [7]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [9]:
query = """CREATE TABLE demo.music_app_history_test (session_id bigint , 
    item_in_session int, 
    artist text, 
    song_title text,
    song_length float, 
    PRIMARY KEY (session_id, item_in_session))"""
session.execute(query);

In [14]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##  Assign the INSERT statements into the `query` variable
        query = "INSERT INTO demo.music_app_history_test (session_id, item_in_session ,artist ,song_title ,song_length)"
        query = query + " VALUES (%s, %s, %s,%s, %s)"
        ##  Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5] )))

In [15]:
query = 'SELECT * FROM demo.music_app_history_test limit 10 '
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.session_id, row.item_in_session, row.artist, row.song_title, row.song_length)


990 1 Frou Frou Breathe In 277.3415832519531
990 2 Justin Timberlake What Goes Around...Comes Around 313.07708740234375
1041 0 David Bowie Sorrow (1997 Digital Remaster) 174.4191436767578
1041 1 J Dilla Won't Do 232.46322631835938
1041 2 Hot Hot Heat In Cairo 295.0264892578125
1041 3 Norther We Rock 236.85179138183594
1041 4 Postal Service Nothing Better (Album) 226.61178588867188
1041 5 311 Something Out Of Nothing 264.5677185058594
1041 6 Jonas Brothers When You Look Me In The Eyes 249.67791748046875
1041 7 Platero Y Tu Cantalojas 368.2999572753906
